## Cleaning the Excel file

In [1]:
import re
import pandas as pd
from datetime import date

from r2e.commom import phone_format
from scripts.utils import COLUMNS_INDEXES, COLUMNS, CENTERS_ID, ASPECTS

file_name = "Brasil-Cesar-2023.xlsx"

df = pd.read_excel(file_name, header=3)

/tmp/ipykernel_266329/29172583.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = df.iloc[:, COLUMNS_INDEXES]
df.rename(columns={df.columns[i]: col_name for i, col_name in COLUMNS.items()}, inplace=True)

In [3]:
def get_center_id(center):
    return CENTERS_ID.get(center, None)

def update_name(name):
    if len(name) > 50:
        name = f"{name[:47]}..."
    return name

def get_aspect(row):
    if pd.notnull(row['p21']) and row['p21'] == "P":
        return ASPECTS.get(row['p21'], None)
    return ASPECTS.get(row['aspect'], None)

def cpf_format(num):
    cpf = str(num)
    cpf = "".join(re.findall(r"\d", num))
    return f"{cpf[:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:]}"

def phone_pre_format(num):
    if len(num) > 20:
        num = f"{num[:20]}"
    return phone_format(num)

def create_id_card(row):
    cpf = row['cpf']
    rg = row['rg']
    ssp = row['ssp']
    
    if isinstance(cpf, str) and pd.notnull(cpf):
        return cpf_format(cpf)
    else:
        if isinstance(rg, str) and isinstance(ssp, str):
            rg = re.sub('[^0-9]', '', rg)
            return f"{rg} | {ssp.upper()}"
        else:
            rg = re.sub('[^0-9]', '', str(rg))
            return rg
        
def update_sos_contact(row):
    sos_contact = str(row['sos_contact']) if pd.notnull(row['sos_contact']) else ""
    sos_type = str(row['sos_type']).lower() if pd.notnull(row['sos_type']) else ""

    if len(sos_contact) > 33:
        sos_contact = f"{sos_contact[:30]}..."
    
    if len(sos_type) > 14:
        sos_type = f"{sos_type[:11]}..."

    new_contact = sos_contact
    if sos_type:
        new_contact = f"{sos_contact} | {sos_type}"
    
    return new_contact

In [4]:
# inserting "center_id" and remove "center_ref"
df['center_id'] = df["center_ref"].apply(get_center_id)
df.drop("center_ref", axis=1, inplace=True)

# name
df['name'] = df['name'].astype(str)
df['name'] = df['name'].apply(update_name)

# removing NaN
df.fillna(
    {
        'p21': '', 
        'city': '', 
        'state': '', 
        'birth': date(2000,1,1), 
        'gender': '', 
        'email': '', 
        'id_card': '',
    }, 
    inplace=True
)

# birth
df['birth'] = pd.to_datetime(df['birth'])
df['birth'] = df['birth'].dt.date

# phone
df['phone'] = df['phone'].astype(str)
df['phone'] = df['phone'].apply(phone_pre_format)

# sos_phone
df['sos_phone'] = df['sos_phone'].astype(str)
df['sos_phone'] = df['sos_phone'].apply(phone_pre_format)

# aspect
df['aspect'] = df.apply(get_aspect, axis=1)
df.drop("p21", axis=1, inplace=True)

# id_card
df['id_card'] = df.apply(create_id_card, axis=1)
df.drop("rg", axis=1, inplace=True)
df.drop("ssp", axis=1, inplace=True)
df.drop("cpf", axis=1, inplace=True)

# sos_contact
df['sos_contact'] = df.apply(update_sos_contact, axis=1)
df.drop("sos_type", axis=1, inplace=True)

In [5]:
people = df.to_dict(orient='records')

# for person in people[40:]:
#     print(person)

## Putting the people dictionary into the Person model

In [6]:
import os
import django

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'r2e.settings')
django.setup()

from apps.person.models import Person

people_with_problems = []
for person in people:
    if person['id_card'] == '':
        person['id_card'] = None
    
    try:
        Person.objects.create(**person)
    except Exception as error:
        person['error'] = str(error)
        people_with_problems.append(person)

In [7]:
errors = pd.DataFrame(people_with_problems)
errors.to_csv('people_with_problems.csv', index=False, encoding="utf-8")
len(people_with_problems)

12

In [8]:
# _file_name = file_name.split(".")
# new_file_name = f"{_file_name[0]}.csv"
# df.to_csv(new_file_name, index=False, encoding="utf-8")